In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S4'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 5 #From 3 to 5 as in paper
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 1912
Total Tokens Found = 24340
Total Tokens Output = 24287
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 1912
Total Tokens= 24287
Unique Tokens= 5182
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  5
Maximum Frequency of Error Token =  3
References Processed= 1912
Sorted Token Size = 5182
Clean Token Size = 2074
*Stop Replacements here
References Processed =  1912
Total Replacement Pairs = 84
Tokens Read = 24287
Tokens Changed =  124
References Changed = 119


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [5]:
import numpy as np

Create a function of the parameters that returns the F score

In [6]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.5,muIncr=0.1,beta=12,sigma=22,epsilon=3.1,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2540
NewMatrix Iterateblocks Total Time = 1.2841967353597283


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2036
NewMatrix Iterateblocks Total Time = 1.2220444260165095


>>>>>>>>>>>>
Starting Iteration mu= 0.7
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1269
NewMatrix Iterateblocks Total Time = 1.006474869325757


>>>>>>>>>>>>
Starting Iteration mu= 0.7999999999999999
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 550
NewMatrix Iterateblocks Total Time = 0.5491659669205546


>>>>>>>>>>>>
Starting Iteration mu= 0.8999999999999999
Starting Iteration epsilon= 3.1
Running NewMatrix Jar!
 Single-Reference Bl

0.91689299

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [14]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [15]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [16]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [17]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


-0.006734025447260872

In [18]:
optimizer=None

In [19]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [20]:
from bayes_opt import SequentialDomainReductionTransformer

In [21]:
bounds_transformer = SequentialDomainReductionTransformer()

In [22]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [23]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(12), 
#             "sigma_sp": inv_softplus(22),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(3.1)},
#     lazy=True,
# )

In [24]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        |  1.999    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        |  1.817    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        |  0.8538   |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        |  0.8812   |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        |  1.385    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  2.159    |  292.7    |  651.6    | -3.872    |  1.119    |  77.52    |
Mu:0.75|Mu+:0.02|Beta:288.00|Sigma:73.00|Epsilon:655.82|EpIter:0.00
|  40       |  2.011    |  288.2    |  655.8    | -3.89     |  1.089    |  73.14    |
Mu:0.75|Mu+:0.02|Beta:288.00|Sigma:73.00|Epsilon:655.40|EpIter:0.00
|  41       |  2.024    |  288.6    |  655.4    | -3.888    |  1.092    |  73.58    |
Mu:0.75|Mu+:0.02|Beta:289.00|Sigma:73.00|Epsilon:655.02|EpIter:0.00
|  42       |  2.018    |  289.0    |  655.0    | -3.886    |  1.095    |  73.97    |
Mu:0.75|Mu+:0.02|Beta:289.00|Sigma:74.00|Epsilon:654.68|EpIter:0.00
|  43       |  2.018    |  289.4    |  654.7    | -3.885    |  1.097    |  74.33    |
Mu:0.75|Mu+:0.02|Beta:289.00|Sigma:74.00|Epsilon:654.37|EpIter:0.00
|  44       |  2.104    |  289.7    |  654.4    | -3.883    |  1.099    |  74.65    |
Mu:0.75|Mu+:0.02|Beta:290.00|Sigma:74.00|Epsilon:654.09|EpIter:0.00
|  45       |  2.142    |  290.0    |  654.1    | -3.882    |  1.101    |  7

In [30]:
optimizer.max

{'target': 2.4044099936553947,
 'params': {'beta_sp': 9.676141279526462,
  'epsilon_sp': 3.1317412419740163,
  'muIncr_logit': -1.9657544206806183,
  'mu_logit': 0.3031309777971007,
  'sigma_sp': 23.599895501271515}}

In [31]:
inv_logit(optimizer.max["target"])

0.91716297

In [32]:
params=optimizer.max["params"]

In [33]:
softplus(params["beta_sp"])

9.676204040779778

In [34]:
softplus(params["sigma_sp"])

23.59989550132784

In [35]:
inv_logit(params["mu_logit"])

0.5752077313781824

In [36]:
inv_logit(params["muIncr_logit"])

0.12284563541222843

In [37]:
softplus(params["epsilon_sp"])

3.174457511837547

In [39]:
entropy_F1(mu=0.575,
           muIncr=0.123,
           beta=9,
           sigma=23,
           epsilon=3.17,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.575
Starting Iteration epsilon= 3.17
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 2439
NewMatrix Iterateblocks Total Time = 1.2227091901004314


>>>>>>>>>>>>
Starting Iteration mu= 0.698
Starting Iteration epsilon= 3.17
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1274
NewMatrix Iterateblocks Total Time = 0.8452738588675857


>>>>>>>>>>>>
Starting Iteration mu= 0.821
Starting Iteration epsilon= 3.17
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 461
NewMatrix Iterateblocks Total Time = 0.4730558041483164


>>>>>>>>>>>>
Starting Iteration mu= 0.944
Starting Iteration epsilon= 3.17
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 100
NewMatrix Iterateblocks Total Time = 0.2266410579904914

Precision = TP / (TP + FP) ......................... = 0.98833139
Recall = TP / (TP + FN) ............................ = 0.85555556
F-

0.91716297